In [ ]:
import pandas as pd
from docx import Document

def extract_tables_from_docx(docx_path):
    doc = Document(docx_path)
    table_data = []

    for table in doc.tables:
        # Extract tables
        headers = [cell.text.strip() for cell in table.rows[0].cells]
        rows = []

        for row in table.rows[1:]:
            row_data = [cell.text.strip() for cell in row.cells]
            rows.append(row_data)

        table_data.append({'Headers': headers, 'Rows': rows})

    return table_data

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text_data = [paragraph.text.strip() for paragraph in doc.paragraphs]

    return text_data

def save_to_excel(table_data, text_data, excel_path):
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for idx, table_info in enumerate(table_data, start=1):
            headers = table_info['Headers']
            rows = table_info['Rows']

            # Ensure headers are unique
            unique_headers = []
            seen_headers = set()

            for header in headers:
                unique_header = header
                counter = 1

                while unique_header in seen_headers:
                    unique_header = f"{header}_{counter}"
                    counter += 1

                unique_headers.append(unique_header)
                seen_headers.add(unique_header)

            df = pd.DataFrame(rows, columns=unique_headers)
            df.to_excel(writer, sheet_name=f'Table_{idx}', index=False)

        text_df = pd.DataFrame({'Text': text_data})
        text_df.to_excel(writer, sheet_name='Text', index=False)

# Example usage
docx_path = 'your_docx_file.docx'
excel_path = 'output_excel_file.xlsx'

table_data = extract_tables_from_docx(docx_path)
text_data = extract_text_from_docx(docx_path)
save_to_excel(table_data, text_data, excel_path)
